In [1]:
!git clone https://github.com/i-am-bee/beeai-framework.git
%cd beeai-framework

Cloning into 'beeai-framework'...
remote: Enumerating objects: 13689, done.
remote: Counting objects: 100% (1767/1767), done.
remote: Compressing objects: 100% (1095/1095), done.
remote: Total 13689 (delta 1129), reused 720 (delta 667), pack-reused 11922 (from 3)
Receiving objects: 100% (13689/13689), 8.00 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (8491/8491), done.
/content/beeai-framework


In [2]:
!pip install beeai openai qiskit python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
from dotenv import load_dotenv


os.environ["IBM_QUANTUM_API_TOKEN"] = "055baaee60c6de48e5f32103bcc6da835d5235529d7c964b8825a30f3c64461ab09a1ca32e642a7acbb01c7cdbea288f4435daac3676841308953097de48bdf7"


os.environ["ARXIV_API_BASE"] = "http://export.arxiv.org/api/query"


with open(".env", "w") as f:
    f.write(f"IBM_QUANTUM_API_TOKEN={os.environ['IBM_QUANTUM_API_TOKEN']}\n")
    f.write(f"ARXIV_API_BASE={os.environ['ARXIV_API_BASE']}\n")


load_dotenv()

print("✅ Loaded tokens:")
print("IBM:", os.getenv("IBM_QUANTUM_API_TOKEN")[:10], "...")
print("arXiv API Base:", os.getenv("ARXIV_API_BASE"))


✅ Loaded tokens:
IBM: 055baaee60 ...
arXiv API Base: http://export.arxiv.org/api/query


In [4]:
!pip install qiskit-ibm-runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 12.1 MB/s eta 0:00:00


In [5]:
import os
from dotenv import load_dotenv
from qiskit_ibm_runtime import QiskitRuntimeService
import requests
import xml.etree.ElementTree as ET

# Load environment variables
load_dotenv()

# === IBM QUANTUM BACKENDS ===
api_token = os.getenv("IBM_QUANTUM_API_TOKEN")

try:
    service = QiskitRuntimeService(channel="ibm_quantum", token=api_token)
    print("🔹 Available IBM Quantum backends:")
    for backend in service.backends():
        print("-", backend.name)
except Exception as e:
    print(f"❌ Error loading IBM Quantum service: {e}")

# === arXiv QUERY (Quantum papers) ===
arxiv_base = os.getenv("ARXIV_API_BASE")
search_query = "quantum+computing"
max_results = 3

arxiv_url = f"{arxiv_base}?search_query=all:{search_query}&start=0&max_results={max_results}"

try:
    response = requests.get(arxiv_url)
    root = ET.fromstring(response.content)

    print("\n🔹 Recent arXiv results (quantum computing):")
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
        link = entry.find('{http://www.w3.org/2005/Atom}id').text.strip()
        print(f"- {title}\n  ↪ {link}")
except Exception as e:
    print(f"❌ Error querying arXiv: {e}")

# === TAVILY WEB SEARCH ===
tavily_key = os.getenv("TAVILY_API_KEY")

if tavily_key:
    print("\n🔹 Tavily Web Search Results (query: quantum computing):")
    tavily_url = "https://api.tavily.com/search"
    headers = {"Content-Type": "application/json"}
    payload = {
        "api_key": tavily_key,
        "query": "quantum computing",
        "search_depth": "basic",
        "include_answer": True,
        "include_sources": True
    }

    try:
        tavily_response = requests.post(tavily_url, json=payload, headers=headers)
        data = tavily_response.json()

        if tavily_response.status_code == 200:
            answer = data.get("answer", "No answer provided.")
            sources = data.get("sources", [])

            print(f"\nTavily Answer:\n{answer}")
            if sources:
                print("\n Sources:")
                for src in sources:
                    title = src.get("title") or "Untitled"
                    url = src.get("url") or "No URL"
                    print(f"- {title} ↪ {url}")
            else:
                print(" No sources were returned by Tavily.")
        else:
            print(f" Tavily API Error {tavily_response.status_code}: {data}")
    except Exception as e:
        print(f"Error during Tavily query: {e}")
else:
    print("\n Tavily API key not found in environment.")


<ipython-input-5-bc049c4fedf3>:14: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService(channel="ibm_quantum", token=api_token)


🔹 Available IBM Quantum backends:
- ibm_brisbane
- ibm_sherbrooke

🔹 Recent arXiv results (quantum computing):
- The Rise of Quantum Internet Computing
  ↪ http://arxiv.org/abs/2208.00733v1
- Unconventional Quantum Computing Devices
  ↪ http://arxiv.org/abs/quant-ph/0003151v1
- Geometrical perspective on quantum states and quantum computation
  ↪ http://arxiv.org/abs/1311.4939v1

 Tavily API key not found in environment.


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Model ID
model_id = "deepseek-ai/deepseek-llm-7b-chat"

# Tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    offload_folder="/content/drive/MyDrive/quantum_tutor_offload",
    torch_dtype="auto"
)
# Create generation pipeline
deepseek_llm = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
!git clone https://github.com/i-am-bee/beeai-platform-agent-starter.git
!git clone https://github.com/i-am-bee/beeai-framework-py-starter.git


Cloning into 'beeai-platform-agent-starter'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 124 (delta 34), reused 20 (delta 14), pack-reused 60 (from 1)
Receiving objects: 100% (124/124), 106.33 KiB | 6.64 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Cloning into 'beeai-framework-py-starter'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 132 (delta 44), reused 53 (delta 35), pack-reused 63 (from 1)
Receiving objects: 100% (132/132), 348.83 KiB | 15.17 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [9]:
# Create project folder in Drive
!mkdir -p /content/drive/MyDrive/quantum_tutor_project

# Move repos into it
!mv beeai-platform-agent-starter /content/drive/MyDrive/quantum_tutor_project/
!mv beeai-framework-py-starter /content/drive/MyDrive/quantum_tutor_project/


mv: inter-device move failed: 'beeai-platform-agent-starter' to '/content/drive/MyDrive/quantum_tutor_project/beeai-platform-agent-starter'; unable to remove target: Directory not empty
mv: inter-device move failed: 'beeai-framework-py-starter' to '/content/drive/MyDrive/quantum_tutor_project/beeai-framework-py-starter'; unable to remove target: Directory not empty


In [10]:
# Paths
agent_dir = "/content/drive/MyDrive/quantum_tutor_project/beeai-platform-agent-starter/agents/quantum_tutor_agent"

# Create the agent folder
!mkdir -p $agent_dir

# Create __init__.py with placeholder class
with open(f"{agent_dir}/__init__.py", "w") as f:
    f.write("""
from beeai.agent import Agent

class QuantumTutorAgent(Agent):
    def run(self, message: str, **kwargs):
        return {"response": f"QuantumTutor received: {message}"}
""")

# Create config.yaml
with open(f"{agent_dir}/config.yaml", "w") as f:
    f.write("""
id: quantum_tutor_agent
class_name: QuantumTutorAgent
module: agents.quantum_tutor_agent
description: Agent that helps users learn Quantum Computing using IBM Qiskit and LLMs.
""")


In [11]:
# Go to the framework starter repo folder
%cd /content/drive/MyDrive/quantum_tutor_project/beeai-framework-py-starter

# Install the framework package locally in editable mode
!pip install -e .


/content/drive/MyDrive/quantum_tutor_project/beeai-framework-py-starter
Obtaining file:///content/drive/MyDrive/quantum_tutor_project/beeai-framework-py-starter
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
import re
import inspect

# --------- Base Agent Definition ---------
class Agent:
    def run(self, message: str, **kwargs):
        raise NotImplementedError

# --------- QuantumTutorAgent Implementation ---------
class QuantumTutorAgent(Agent):
    def __init__(self, llm_pipeline):
        self.llm = llm_pipeline

    def format_response(self, response_text):
        # Add emojis to key quantum terms for fun and engagement
        response_text = re.sub(r'\bquantum\b', 'Quantum', response_text, flags=re.IGNORECASE)
        response_text = re.sub(r'\bentanglement\b', 'entanglement 🔗', response_text, flags=re.IGNORECASE)
        return response_text

    def classify_query(self, message: str) -> str:
        lowered = message.lower()
        if any(k in lowered for k in ['code', 'python', 'program', 'implementation']):
            return 'code'
        elif any(k in lowered for k in ['arxiv', 'paper', 'research', 'journal', 'citation']):
            return 'research'
        elif any(k in lowered for k in ['difference', 'vs', 'compare']):
            return 'comparison'
        elif any(k in lowered for k in ['formula', 'derive', 'equation', 'proof', 'math']):
            return 'math'
        elif any(k in lowered for k in ['application', 'real world', 'industry']):
            return 'application'
        elif any(k in lowered for k in ['history', 'who discovered', 'origin']):
            return 'history'
        elif any(k in lowered for k in ['fun fact', 'joke', 'trivia']):
            return 'fun'
        elif any(k in lowered for k in ['mcq', 'quiz', 'questionnaire']):
            return 'quiz'
        elif any(k in lowered for k in ['translate', 'in hindi', 'in tamil', 'meaning in']):
            return 'translation'
        else:
            return 'general'

    def build_prompt(self, message: str, category: str) -> str:
        base = f"""
        You are QuantumTutor, a friendly and enthusiastic tutor specializing in quantum computing.
        Explain the following query in a simple, fun, and structured way: '{message}'.
        Use analogies, real-world examples, and avoid complex jargon to make it learner-friendly.
        Structure your response with:
        - An introduction (start with a hook to grab attention)
        - Key points using bullet points (•) to break down the concept
        - A conclusion (wrap up with an encouraging note)
        """

        if category == 'code':
            base += "\nAlso include Python code snippets if relevant."
        elif category == 'research':
            base += "\nProvide arXiv links to relevant research papers if applicable."
        elif category == 'comparison':
            base += "\nCompare the items side-by-side with advantages and disadvantages."
        elif category == 'math':
            base += "\nInclude math derivations or formulas as needed."
        elif category == 'application':
            base += "\nEmphasize real-world applications in industry or research."
        elif category == 'history':
            base += "\nAdd historical background and discovery timeline."
        elif category == 'fun':
            base += "\nInclude a fun fact or a joke related to the topic."
        elif category == 'quiz':
            base += "\nAdd 2-3 MCQs with answers."
        elif category == 'translation':
            base += "\nTry translating the key idea to Hindi or regional languages."

        return base

    def clean_prompt_echo(self, text):
        lines = text.strip().splitlines()
        cleaned = "\n".join(
            line for line in lines
            if not re.match(r'^\s*(you are|structure|explain|use|avoid|start|wrap)', line.strip(), re.IGNORECASE)
        )
        return cleaned.strip()

    def run(self, message: str, **kwargs):
        category = self.classify_query(message)
        prompt = self.build_prompt(message, category)
        generated = self.llm(prompt, max_new_tokens=2000, do_sample=True)
        full_output = generated[0]['generated_text']

        # Trim at common starting keywords
        start_keywords = ["introduction:", "let's dive in", "here's what you need to know", "in simple terms"]
        trimmed = None
        for keyword in start_keywords:
            idx = full_output.lower().find(keyword)
            if idx != -1:
                trimmed = full_output[idx:]
                break

        if not trimmed:
            trimmed = full_output

        cleaned = self.clean_prompt_echo(trimmed)
        formatted_response = self.format_response(cleaned)
        return {'response': formatted_response}


# -------- Save Final QuantumTutorAgent into __init__.py --------
code = """import re

# --------- Base Agent Definition ---------
class Agent:
    def run(self, message: str, **kwargs):
        raise NotImplementedError

# --------- QuantumTutorAgent Implementation ---------
class QuantumTutorAgent(Agent):
    def __init__(self, llm_pipeline):
        self.llm = llm_pipeline

    def format_response(self, response_text):
        # Add emojis to key quantum terms for fun and engagement
        response_text = re.sub(r'\\bquantum\\b', 'Quantum', response_text, flags=re.IGNORECASE)
        response_text = re.sub(r'\\bentanglement\\b', 'entanglement 🔗', response_text, flags=re.IGNORECASE)
        return response_text

    def classify_query(self, message: str) -> str:
        lowered = message.lower()
        if any(k in lowered for k in ['code', 'python', 'program', 'implementation']):
            return 'code'
        elif any(k in lowered for k in ['arxiv', 'paper', 'research', 'journal', 'citation']):
            return 'research'
        elif any(k in lowered for k in ['difference', 'vs', 'compare']):
            return 'comparison'
        elif any(k in lowered for k in ['formula', 'derive', 'equation', 'proof', 'math']):
            return 'math'
        elif any(k in lowered for k in ['application', 'real world', 'industry']):
            return 'application'
        elif any(k in lowered for k in ['history', 'who discovered', 'origin']):
            return 'history'
        elif any(k in lowered for k in ['fun fact', 'joke', 'trivia']):
            return 'fun'
        elif any(k in lowered for k in ['mcq', 'quiz', 'questionnaire']):
            return 'quiz'
        elif any(k in lowered for k in ['translate', 'in hindi', 'in tamil', 'meaning in']):
            return 'translation'
        else:
            return 'general'

    def build_prompt(self, message: str, category: str) -> str:
        base = f\"\"\"
        You are QuantumTutor, a friendly and enthusiastic tutor specializing in quantum computing.
        Explain the following query in a simple, fun, and structured way: '{message}'.
        Use analogies, real-world examples, and avoid complex jargon to make it learner-friendly.
        Structure your response with:
        - An introduction (start with a hook to grab attention)
        - Key points using bullet points (•) to break down the concept
        - A conclusion (wrap up with an encouraging note)
        \"\"\"

        if category == 'code':
            base += "\\nAlso include Python code snippets if relevant."
        elif category == 'research':
            base += "\\nProvide arXiv links to relevant research papers if applicable."
        elif category == 'comparison':
            base += "\\nCompare the items side-by-side with advantages and disadvantages."
        elif category == 'math':
            base += "\\nInclude math derivations or formulas as needed."
        elif category == 'application':
            base += "\\nEmphasize real-world applications in industry or research."
        elif category == 'history':
            base += "\\nAdd historical background and discovery timeline."
        elif category == 'fun':
            base += "\\nInclude a fun fact or a joke related to the topic."
        elif category == 'quiz':
            base += "\\nAdd 2-3 MCQs with answers."
        elif category == 'translation':
            base += "\\nTry translating the key idea to Hindi or regional languages."

        return base

    def clean_prompt_echo(self, text):
        lines = text.strip().splitlines()
        cleaned = "\\n".join(
            line for line in lines
            if not re.match(r'^\\s*(you are|structure|explain|use|avoid|start|wrap)', line.strip(), re.IGNORECASE)
        )
        return cleaned.strip()

    def run(self, message: str, **kwargs):
        category = self.classify_query(message)
        prompt = self.build_prompt(message, category)
        generated = self.llm(prompt, max_new_tokens=2000, do_sample=True)
        full_output = generated[0]['generated_text']

        start_keywords = ["introduction:", "let's dive in", "here's what you need to know", "in simple terms"]
        trimmed = None
        for keyword in start_keywords:
            idx = full_output.lower().find(keyword)
            if idx != -1:
                trimmed = full_output[idx:]
                break

        if not trimmed:
            trimmed = full_output

        cleaned = self.clean_prompt_echo(trimmed)
        formatted_response = self.format_response(cleaned)
        return {'response': formatted_response}
"""

file_path = '/content/drive/MyDrive/quantum_tutor_project/beeai-platform-agent-starter/agents/quantum_tutor_agent/__init__.py'
with open(file_path, 'w') as f:
    f.write(code)

print(f"✅ QuantumTutorAgent saved successfully to: {file_path}")


✅ QuantumTutorAgent saved successfully to: /content/drive/MyDrive/quantum_tutor_project/beeai-platform-agent-starter/agents/quantum_tutor_agent/__init__.py


In [13]:
!pip install transformers

In [1]:
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Mount drive only once
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Add project path
sys.path.append('/content/drive/MyDrive/quantum_tutor_project/beeai-platform-agent-starter')

# Load Deepseek model
def load_llm():
    model_id = "deepseek-ai/deepseek-llm-7b-chat"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        offload_folder="/content/drive/MyDrive/quantum_tutor_offload",
        torch_dtype="auto"
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load the agent
def load_agent(llm_pipeline):
    from agents.quantum_tutor_agent import QuantumTutorAgent
    return QuantumTutorAgent(llm_pipeline)

# Execute once
if __name__ == "__main__":
    print("🚀 Initializing Deepseek model...")
    llm = load_llm()
    print("✅ Model loaded.")

    print("🔁 Loading QuantumTutorAgent...")
    agent = load_agent(llm)
    print("✅ Agent ready!")

    query = "Tell me about IBM quantum backends and latest research papers."
    result = agent.run(query)
    print("\n🤖 Agent response:\n", result["response"])


Mounted at /content/drive
🚀 Initializing Deepseek model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Model loaded.
🔁 Loading QuantumTutorAgent...
✅ Agent ready!

🤖 Agent response:
 Introduction:

IBM Quantum backends are like the superheroes of the Quantum computing world. They help us store and process Quantum information securely and efficiently. Think of them as the behind-the-scenes heroes who make our Quantum computers work smoothly.

Key points:

1. IBM Quantum Backends:
   • Imagine you are a superhero, and your job is to keep the Quantum world safe and organized.
   • The backends are like your secret headquarters, where you keep all your Quantum tools, information, and data.
   • They help us access Quantum computing resources remotely, like calling upon a superhero to help with a problem.

2. Latest Research Papers:
   • Quantum Information Processing (QIP) is a scientific journal where researchers share their latest findings on Quantum computing.
   • Some recent research papers are available on the arXiv website.
   • For example, "A Scalable Quantum Algorithm for the Ma

In [5]:
# Test it
response = agent.run("Teach me Quantum Computing assuming,I WERE A 5 YEAR OLD ")
print(response['response'])

Introduction:
Quantum Computing is a super cool way to solve problems! Imagine it as your super-powered brain that can solve complex problems faster than a regular computer.

Key Points (•):

1. **What is Quantum Computing?** (Analogies):
    - Just like how your brain can process multiple thoughts at once, Quantum computers can process many pieces of information at the same time.
    - Think of it like a Rubik's Cube. Your brain can solve one at a time, but a Quantum computer can solve many at once.

2. **Bits vs Qubits** (Analogies):
    - In regular computers, we use bits, which are like tiny light switches that either turn on or off.
    - Quantum computers use qubits, which are like tiny light switches that can be in any state between on and off at the same time!

3. **entanglement 🔗** (Analogies):
    - Imagine your favorite toy blocks. Each block represents a qubit, and they can be in any state.
    - Now, imagine you can connect these blocks together in a way that their states 

In [8]:
# Test it
response = agent.run(" How does quantum field theory reconcile particle and wave behavior?  IBM")
print(response['response'])

Introduction:
        Quantum Field Theory (QFT) is a fascinating concept that helps us understand the behavior of particles and waves. Picture it as a giant puzzle, where each piece represents a Quantum field. These fields are everywhere in the universe and are responsible for giving particles their properties, such as mass and charge. Now, imagine you're a detective trying to solve a mystery!

        Key points:
        • • • • • • •
        1. Quantum fields contain energy and particles, like a big cosmic soup.
        2. The energy in these fields can be in different forms, such as particles or waves.
        3. Just like how you can change a liquid into a solid or a gas, these fields can change their energy forms.
        4. When the energy changes form, it creates particles that we observe in our world.
        5. These particles can have different properties, such as mass, charge, and spin.
        6. Particle and wave behavior are not mutually exclusive. You can have both at t

In [9]:
# Test it
response = agent.run(" How does quantum field theory reconcile particle and wave behavior?arXiv")
print(response['response'])

Introduction:
Hello little learners! Today, we're going to talk about a big, fascinating idea called Quantum Field Theory. It's like a puzzle that helps us understand how tiny particles like electrons and photons behave. We'll use fun examples to help you understand!

Key Points:

1. • A particle is like a tiny Lego block, while a wave is like the ripples it makes on water.
2. • Quantum Field Theory is like a big playground where these particles and waves play together.
3. • Quantum Field Theory explains how particles can behave like both a Lego block and a ripple at the same time, which is called wave-particle duality.
4. • A famous experiment called the double-slit experiment shows this wave-particle duality in action.
5. • To reconcile these two seemingly different behaviors, Quantum Field Theory says that particles have energy and charge, which can be both particle-like and wave-like.

Conclusion:
Quantum Field Theory is like a magic trick that helps us understand how particles can

In [10]:
# Test it
response = agent.run("write a qiskit code to generate bell state  IBM")
print(response['response'])

Introduction:
As a Quantum tutor, I often get asked to write a code in Qiskit that generates a Bell state. A Bell state is a special kind of Quantum state that is very important in Quantum computing. It's like a secret handshake between qubits, and it's crucial for Quantum communication and computing. To explain this concept, let's imagine that the Bell state is like a secret message that only two people can understand. They can't share the message with anyone else, and if they try to copy the message, it will change. That's why it's so important in Quantum computing!

Key points (•):

1. A Bell state is a special kind of Quantum state that can only be understood by two qubits.
2. To create a Bell state, you need to use a Quantum gate called the controlled-NOT gate (CNOT).
3. The CNOT gate works by measuring the control qubit and then applying a NOT gate to the target qubit based on the result of the measurement.
4. In Qiskit, you can create a Bell state by using the `QuantumRegister` 

In [2]:
# Test it
response = agent.run("What is quantum superposition, and how does it enable quantum parallelism?")
print(response['response'])

Introduction:
        Quantum superposition is a fundamental principle in Quantum mechanics that allows particles to exist in multiple states at once. Imagine a coin that is spinning in mid-air, it can be considered as being in two states - heads or tails - until it lands. Similarly, a Quantum particle can be in multiple states at the same time.

        Key points:
        • Quantum superposition allows a Quantum particle to be in multiple states at once, similar to the spinning coin example.
        • Quantum parallelism is the ability of a Quantum system to perform multiple calculations simultaneously, utilizing the principle of superposition.
        • In a Quantum computer, Quantum bits (qubits) can exist in multiple states at once, enabling them to perform multiple calculations simultaneously.
        • By using Quantum superposition, a Quantum computer can solve complex problems much faster than a classical computer.
        

        Conclusion:
        Quantum superposition is

In [3]:
# Test it
response = agent.run("What problem does Grover’s algorithm solve more efficiently than classical methods?")
print(response['response'])

Introduction:
        Imagine you're looking for a lost item in a messy room. You can search for the item using your eyes, but it takes a lot of time. Now, imagine you have a special tool that can help you find the item quickly - that's like using Grover's algorithm!

        Key points:
        • Grover's algorithm is a Quantum algorithm, which means it uses principles of Quantum mechanics to solve problems.
        • The problem that Grover's algorithm solves more efficiently than classical methods is searching for a specific item (or state) in an unsorted database.
        • Classical methods (like using your eyes) would take O(N) time to find the item, where N is the number of items in the database.
        • Grover's algorithm, on the other hand, can find the item in O(sqrt(N)) time, which is much faster!
        • Imagine if you could find the item in just a few steps, rather than having to look at every item in the room. That's how Grover's algorithm helps you find the item more

In [4]:
# Test it
response = agent.run("How does the Bell inequality test the completeness of quantum mechanics")
print(response['response'])

Introduction:
        Quantum mechanics is a fascinating field that has been studied for decades, and its theories continue to be tested and refined. One of the key aspects of Quantum mechanics that is often put to the test is its completeness. The Bell inequality is a test that is used to determine whether or not Quantum mechanics is complete. In this response, we will explore how the Bell inequality tests the completeness of Quantum mechanics using analogies, real-world examples, and avoiding complex jargon.
        
        Key points:
        • The Bell inequality is a mathematical expression that describes the limits of the predictions of Quantum mechanics.
        • The test involves measuring the properties of pairs of particles that have interacted at a distance, such as photons.
        • If the results of the measurements match the predictions of Quantum mechanics, then it is likely that Quantum mechanics is complete.
        • However, if the results do not match the predict

In [5]:
# Test it
response = agent.run("What is quantum superposition, and how does it enable quantum parallelism?")
print(response['response'])

Introduction:
        Quantum superposition is a fundamental principle in Quantum mechanics that allows Quantum particles to exist in multiple states at once. Imagine your favorite superhero, Spiderman, who can swing between different buildings without actually being in two places at once. Just like Spiderman, Quantum particles can be in multiple places simultaneously until they are observed or measured. This phenomenon enables Quantum parallelism, which allows Quantum computers to perform many calculations simultaneously, making them much faster and more powerful than classical computers.

        Key points:
        • Quantum superposition allows particles to be in multiple states at once.
        • This is similar to how Spiderman can be swinging between multiple buildings at the same time.
        • Quantum parallelism is the ability of Quantum computers to perform many calculations simultaneously.
        • Quantum computers can be up to 100 trillion times faster than classical co

In [6]:
# Test it
response = agent.run("Iluustrate comparision between Qunatum Machine Learning and Machine Learning?")
print(response['response'])

Introduction:
        Quantum computers have revolutionized the world of computing, and with them comes the emergence of Quantum machine learning. While both machine learning and Quantum machine learning are related fields, they have their unique differences. In this explanation, we will explore the comparison between Quantum machine learning and traditional machine learning, and help you understand the fundamentals of each.
        
        Key Points:
        • Quantum machine learning is a field that uses Quantum computers to perform machine learning tasks.
        • Traditional machine learning, on the other hand, uses classical computers to perform machine learning tasks.
        • Quantum computers have the potential to solve complex problems that classical computers struggle with.
        • The main difference between Quantum machine learning and traditional machine learning is that Quantum machine learning uses Quantum algorithms to solve problems.
        • Quantum computers u

In [7]:
# Test it
response = agent.run("Could you help me understand Quantum SYM's key idea")
print(response['response'])

Introduction:
        "Quantum SYM is a fascinating concept in the world of Quantum computing, and I'm here to help you understand it. Imagine you're trying to solve a complex puzzle with many pieces, and you need a special tool to make it easier. That's what Quantum SYM is like - it's a tool that makes Quantum computing more efficient and easier to understand. Let's dive in!"

        Key points:
        • Quantum SYM stands for "Supersymmetric Quantum Mechanics," a theoretical framework that aims to combine Quantum mechanics and general relativity.
        • It's like a super hero that can help you solve complex problems in physics and Quantum computing.
        • In this framework, the Quantum world is described by two equations: one for bosons (particles that can occupy the same state) and one for fermions (particles that have special properties that prevent them from occupying the same state).
        • By introducing extra symmetries, Quantum SYM helps us understand the behavior 

33+26+27+39 = 31.25s


18+24+25+18 = 21.25s (Qs from RAG agent)